# Servo Motor Control mit Raspberry Pi und Jupyter Notebook

## 1. Einführung
Dieses Jupyter Notebook führt in die praktische Anwendung der Steuerung einer LED, eines Buzzers und eines BME680-Sensors mit einem Raspberry Pi ein. 
Es kombiniert Elektronik und Python-Programmierung, um eine Überwachung der Temperatur zu erstellen und bei Überschreitung eines Schwellenwerts ein SOS-Signal zu senden. 
Das Projekt zeigt, wie man ein digitales Signal einsetzt, um die Funktionalität verschiedener Hardware-Komponenten zu steuern.

### Projektziel
Das Ziel dieses Projekts ist es, eine LED und einen Buzzer basierend auf der Temperaturüberwachung eines BME680-Sensors zu steuern und bei bestimmter Temperatur ein SOS-Signal zu senden.

### Relevanz
Dieses Projekt ist relevant, da es die Grundlagen der Elektronik und Programmierung kombiniert und ein grundlegendes Verständnis 
für die Steuerung von Aktoren in Abhängigkeit von Sensordaten entwickelt.


## 2. Grundlagen und Theorie

### BME680 Sensor
- **Funktionsweise:** Der BME680 ist ein Umweltsensor, der Temperatur, Luftfeuchtigkeit, Druck und Luftqualität misst.
- **Steuerung:** Die Kommunikation mit dem BME680 erfolgt über das I2C-Protokoll.
- [Datasheet](https://cdn-shop.adafruit.com/product-files/3660/BME680.pdf) 

## Buzzer

- **Funktionsweise:** Ein Buzzer ist ein elektronisches Bauteil, das einen akustischen Ton erzeugt, wenn Strom durch ihn fließt. 
- **Steuerung:** In diesem Projekt verwenden wir einen passiven Buzzer, der über ein PWM-Signal (Pulsweitenmodulation) gesteuert wird, um verschiedene Töne zu erzeugen. Die Frequenz des PWM-Signals bestimmt den Ton des Buzzers.

**PWM-Signale für Buzzer**

- **Definition:** Wie bei Servomotoren wird auch beim Buzzer PWM verwendet, um das Signal zu modulieren. Hier variiert jedoch die Frequenz  des PWM-Signals, um unterschiedliche Töne zu erzeugen.
- **Anwendung:** Der Buzzer in diesem Projekt wird mit einer PWM-Frequenz von 1000 Hz angesteuert, um einen konstanten Ton zu erzeugen.

### GPIO-Steuerung
- **Definition:** GPIO steht für General Purpose Input/Output. Die Pins des Raspberry Pi können entweder als Eingabe- oder Ausgabe-Pins konfiguriert werden.
- **PWM-Anwendung:** PWM steht für Pulsweitenmodulation und wird hier verwendet, um die Lautstärke des Buzzers zu steuern.  

### SOS Signal
- **Morsecode:** SOS ist ein internationaler Notruf im Morsecode, dargestellt durch drei kurze Signale (Punkte), gefolgt von drei langen Signalen (Striche) und wiederum drei kurzen Signalen (Punkte).  
- [Für weitere über Morsecode klicken Sie hier](https://morsedecoder.com/de/)  



## 3. Materialien und Werkzeuge

### Software
- Raspbian OS 
- Python 3
- Jupyter Notebook

### Hardware
- Raspberry Pi 5
- Breadboard
- Jumper-Kabel
- BME680 Sensor
- LED
- Buzzer
- 5V Stromversorgung

### Verdrahtung
- Die LED wird mit dem GPIO 17 des Raspberry Pi verbunden.
- Der Buzzer wird mit dem GPIO 18 des Raspberry Pi verbunden.
- Der BME680 Sensor wird über I2C mit dem Raspberry Pi verbunden.


## 4. Schaltungsdesign

### Raspberry Pi 5 GPIO-Pinout
<img src="Bilder/Raspberry_Pi_5_GPIO-Pinout.png" alt="Rasp Pi Pinout" style="width:400px;">   
[Quelle](https://community.particle.io/t/following-getting-started-guide-for-raspberry-pi-how-do-i-use-analog-pins/27771/2)  

### Zu BME680 Sensor
![BME680 Schaltung](Bilder/BME680.jpg)  
[Quelle](https://blog.adafruit.com/2017/11/09/new-product-adafruit-bme680-temperature-humidity-pressure-and-gas-sensor/)  

### Schaltplan
![LED und Buzzer Schaltung](Bilder/LED_Buzzer_Schaltung.png)  
![Standard Bild fehlt noch](Bilder/)  


## 5. Implementierung

### Hardware-Aufbau
1. Komponenten auf dem Breadboard platzieren und verkabeln.
2. Raspberry Pi aufstellen und mit Strom versorgen.

### Software-Setup
1. Raspbian OS installieren.
2. Python und Jupyter Notebook installieren:
    ```sh
    sudo apt-get update
    sudo apt-get install python3 jupyter
    ```

### Installiere die erforderlichen Bibliotheken:
```sh
sudo pip3 install adafruit-circuitpython-bme680
sudo apt-get install python3-rpi.gpio
```


In [ ]:
import board
import busio
import adafruit_bme680
import RPi.GPIO as GPIO
import time
import sys

# GPIO Pins definieren
LED_PIN = 17
BUZZER_PIN = 18

# GPIO-Modus (BCM) und Pins konfigurieren
GPIO.setmode(GPIO.BCM)
GPIO.setup(LED_PIN, GPIO.OUT)
GPIO.setup(BUZZER_PIN, GPIO.OUT)

# PWM für den Buzzer einrichten
pwm = GPIO.PWM(BUZZER_PIN, 1000)  # 1000 Hz Frequenz
pwm.start(0)  # Start PWM mit 0% Duty Cycle (Buzzer aus)

# I2C initialisieren
i2c = busio.I2C(board.SCL, board.SDA)

# BME680 Sensor initialisieren
try:
    bme680 = adafruit_bme680.Adafruit_BME680_I2C(i2c)
    bme680.sea_level_pressure = 1013.25
    print("Sensor erfolgreich initialisiert!")
except Exception as e:
    print(f"Fehler beim Initialisieren des Sensors: {e}")
    GPIO.cleanup()
    exit(1)

def dot():
    GPIO.output(LED_PIN, GPIO.HIGH)
    pwm.ChangeDutyCycle(50)  # Buzzer an
    time.sleep(1)  # LED 1 Sekunde an (Punkt)
    GPIO.output(LED_PIN, GPIO.LOW)
    pwm.ChangeDutyCycle(0)  # Buzzer aus
    time.sleep(1)  # LED 1 Sekunde aus (Pause zwischen Zeichen)

def dash():
    GPIO.output(LED_PIN, GPIO.HIGH)
    pwm.ChangeDutyCycle(50)  # Buzzer an
    time.sleep(3)  # LED 3 Sekunden an (Strich)
    GPIO.output(LED_PIN, GPIO.LOW)
    pwm.ChangeDutyCycle(0)  # Buzzer aus
    time.sleep(1)  # LED 1 Sekunde aus (Pause zwischen Zeichen)

def letter_space():
    time.sleep(3)  # Pause zwischen Buchstaben (3 Sekunden)

def word_space():
    time.sleep(7)  # Pause zwischen Wörtern (7 Sekunden)

def send_sos():
    dot(); dot(); dot()  # S
    letter_space()
    dash(); dash(); dash()  # O
    letter_space()
    dot(); dot(); dot()  # S
    word_space()

try:
    while True:
        # Temperatur lesen
        temperature = bme680.temperature
        if temperature is not None:
            # Temperatur ausgeben und Konsole aktualisieren
            sys.stdout.write(f"\rAktuelle Temperatur: {temperature:.2f} °C")
            sys.stdout.flush()
            
            # Temperaturüberprüfung
            if temperature >= 30:
                # Temperatur >= 30°C: LED und Buzzer einschalten
                GPIO.output(LED_PIN, GPIO.HIGH)
                pwm.ChangeDutyCycle(50)  # Buzzer an
                send_sos()  # SOS-Morsecode senden
            else:
                # Temperatur < 30°C: LED und Buzzer ausschalten
                GPIO.output(LED_PIN, GPIO.LOW)
                pwm.ChangeDutyCycle(0)  # Buzzer aus
        
        else:
            print("Fehler beim Lesen der Temperatur")
        
        # Verzögerung vor der nächsten Messung
        time.sleep(1)

except KeyboardInterrupt:
    print("\nProgramm beendet")

finally:
    pwm.stop()  # Stoppe PWM
    GPIO.cleanup()  # GPIO-Pins zurücksetzen


## 6. Experimente und Ergebnisse

### Versuchsaufbau
- Der oben beschriebene Hardware-Aufbau wurde verwendet.
- Der Code wurde in einem Jupyter Notebook ausgeführt.
- Der Sensor überwachte die Temperatur, und die LED und der Buzzer reagierten entsprechend.


## 7. Diskussion und Fazit

### Diskussion
Das Projekt hat gezeigt, wie man eine LED und einen Buzzer mit einem Raspberry Pi und Python basierend auf Sensordaten steuert. 
Jupyter Notebooks wurden verwendet, um die Hardware-Experimente zu entwickeln und zu testen.

### Verbesserungsmöglichkeiten
- Erweiterung auf komplexere Bewegungsmuster und Sensordatenverarbeitungen.
- Implementierung eines Rückkopplungssystems zur Positionsbestimmung.

### Fazit
Das Projekt war erfolgreich in der Demonstration der Grundlagen der LED- und Buzzersteuerung mit einem Raspberry Pi und der Nutzung von Jupyter Notebooks. 
Es bietet eine solide Grundlage für weiterführende Projekte in der Robotik und Automatisierung.
